____________
## 3 - Get ratio of Sentinel-1 pixel intensities for AOI

In [ ]:
# Define function for retrieving satellite imagery from Descartes platform and displaying on map
def getImage(satellite,bands,st_date,end_date,time,opacity=1,mapNum=m1,cloudFraction=0.1):
    img = wf.ImageCollection.from_id(satellite,start_datetime=st_date[time], end_datetime=end_date[time])
    if 'sentinel-2' in satellite: # Use sentinel cloud-mask band if available
        img = img.filter(lambda img: img.properties["cloud_fraction"] <= cloudFraction)
        img = img.map(lambda img: img.mask(img.pick_bands('cloud-mask')==1))
    mos = (img.mosaic().pick_bands(bands))
    globals()['mos_'+str(time+1)+str(bands)] = mos
    display = mos.visualize('Image '+str(time+1)+str(bands), map=mapNum)
    display.opacity = opacity

In [ ]:
# Initialise map
m2 = wf.interactive.MapApp()
m2.center, m2.zoom = (lat, lon), zoom

satellite, bands = "sentinel-1:GRD", ['vh','vv']
st_date, end_date = ['2020-07-30T00:00:00.000Z', '2020-08-05T00:00:00.000Z'], ['2020-07-30T14:00:00.000Z', '2020-08-05T14:00:00.000Z'] # Timeframes for before-after imagery: start 1, start 2; end 1 ,end 2

# Loop over dates (time 1 & time 2)
for i in range(len(st_date)):
    getImage(satellite, bands, st_date, end_date, i, 0, m2)
    for j in bands:
        getImage(satellite, j, st_date, end_date, i, 0, m2)
    
# Calculate logarithmic ratio for RGB images and display
ratio = wf.log10(globals()['mos_1'+str(bands)] / globals()['mos_2'+str(bands)])
rdisplay = ratio.visualize('Ratio' ,map=m2)
rdisplay.opacity = 1
m2.add_layer(testPoly)
m2

In [ ]:
# Area of interest defined by test polygon
AOI = dl.scenes.AOI(bounds = (testPoly.locations[0][0]['lng'], testPoly.locations[0][0]['lat'], testPoly.locations[0][2]['lng'], testPoly.locations[0][2]['lat']),
                    resolution=20,
                    crs=m2.geocontext().crs)

# Compute values from ratio within AOI
change = ratio.compute(geoctx = AOI)

In [ ]:
# Assign initial beliefs to each pixel
bounds = AOI.bounds
longs, lats = np.linspace(bounds[0],bounds[2],change.geocontext['arr_shape'][1]), np.linspace(bounds[3],bounds[1],change.geocontext['arr_shape'][0])
xm, ym = np.meshgrid(longs,lats)
initial = gpd.GeoDataFrame(pd.DataFrame(np.concatenate((np.ones([len(longs)*len(lats),2])*0.5, change.ndarray[0].reshape(-1,1), change.ndarray[1].reshape(-1,1)), axis=1), columns = ['noDmg','dmg','vh','vv']), geometry=gpd.points_from_xy(xm.reshape(-1), ym.reshape(-1)),crs={'init': 'epsg:4326'})